In [141]:
import json

def collect_cz_blocks(file_path):
    """
    Reads a Cirq circuit (JSON) from 'file_path' and groups consecutive CZ gates
    under the rule:

      - We read all CZ gates G1, G2, G3, ... in chronological order.
      - Maintain an in-progress 'current block' of multiple CZ gates.
      - For each new CZ gate G_new, compare it with *all* gates in the current block.
         -> If they share qubits AND there is a single-qubit gate on those shared qubits
            in the intervening moments, we finalize the block and start a *new* block
            with G_new.
         -> Otherwise (no conflict with *all* gates in the block), we add G_new to the
            current block.
      - Continue until we've processed all CZ gates.

    Returns:
      A list of blocks, each block is a list of (q1, q2) pairs representing
      the qubits for each CZ in that block.

    This matches the user's requirement that any new CZ must be 'compatible'
    with *every* gate in the block (no single-qubit ops on shared qubits in between).
    If there's a conflict with even one gate in the block, we break and start a new block.
    """
    with open(file_path, "r") as f:
        circuit_json = json.load(f)

    moments = circuit_json["moments"]
    
    # 1) Collect all CZ gates in chronological order
    #    We'll store them as a list of (moment_index, frozenset_of_qubits)
    cz_list = []
    for m_idx, moment in enumerate(moments):
        for op in moment["operations"]:
            if op["gate"]["cirq_type"] == "CZPowGate":
                qubits_used = [q["name"] for q in op["qubits"]]
                if len(qubits_used) == 2:
                    cz_list.append((m_idx, frozenset(qubits_used)))
    
    if not cz_list:
        return []  # No CZ gates => no blocks

    # Sort by moment index just in case
    cz_list.sort(key=lambda x: x[0])

    # Helper function: check for single-qubit gates on a set of qubits
    # in the *strictly between* moments [start_m+1 ... end_m-1].
    def has_single_qubit_on(qubits_of_interest, start_m, end_m):
        if start_m > end_m:
            return False
        for m_idx_check in range(start_m, end_m + 1):
            for op in moments[m_idx_check]["operations"]:
                qbs = [q["name"] for q in op["qubits"]]
                if len(qbs) == 1:
                    # single-qubit gate on qbs[0]
                    if qbs[0] in qubits_of_interest:
                        return True
        return False

    blocks = []          # final list of blocks
    current_block = []   # list of (q1, q2) for the current block
    # We'll also store the moment indices in parallel so we can check conflicts
    current_block_moments = []  # list of moment_idx for each gate in the block

    # Start with the first CZ gate in its own block
    first_m_idx, first_qubits = cz_list[0]
    current_block = [tuple(first_qubits)]
    current_block_moments = [first_m_idx]

    # 2) Process each subsequent CZ gate
    for i in range(1, len(cz_list)):
        new_m_idx, new_qubits = cz_list[i]
        
        # We'll compare new_qubits with *all* gates in current_block
        # If there's a conflict with *any*, we finalize & start new
        conflict_found = False

        for j, old_qubits_tuple in enumerate(current_block):
            old_qubits = frozenset(old_qubits_tuple)
            old_m_idx = current_block_moments[j]

            # Find shared qubits
            shared = old_qubits.intersection(new_qubits)
            if not shared:
                # If no overlap, no possibility of conflict with *this* gate
                continue

            # If they share qubits, check for single-qubit gates on that shared set
            # in the strictly between moments: (old_m_idx+1) ... (new_m_idx-1).
            start_m = old_m_idx + 1
            end_m = new_m_idx - 1

            if has_single_qubit_on(shared, start_m, end_m):
                conflict_found = True
                break

        if conflict_found:
            # We must finalize the current block
            blocks.append(current_block)
            # Start a new block with this new CZ
            current_block = [tuple(new_qubits)]
            current_block_moments = [new_m_idx]
        else:
            # No conflict with *all* gates in the current block => extend the block
            current_block.append(tuple(new_qubits))
            current_block_moments.append(new_m_idx)

    # 3) After the loop, finalize the last block
    if current_block:
        blocks.append(current_block)

    return blocks




In [161]:
if __name__ == "__main__":
    path = "circuit4.json"
    blocks = collect_cz_blocks(path)

    print("Final blocks of CZ gates:")
    for i, block in enumerate(blocks, start=1):
        print(f"Block {i}:", block)

Final blocks of CZ gates:
Block 1: [('q_0', 'q_3'), ('q_0', 'q_6')]
Block 2: [('q_4', 'q_3'), ('q_0', 'q_1'), ('q_5', 'q_3'), ('q_7', 'q_6'), ('q_0', 'q_2'), ('q_8', 'q_6')]


if cz node inline with other cz node and no gate between them
    if 2nd nodes are not inline
    elif 2nd nodes are in line but there is no gate between them
    group CZ0 with CZ1 in one block.
    do this for next CZ gate and end when the above is not true
    start a new block and start with CZi where i is the number of gate that ended the last block

In [153]:
# Extract gate name, exponent, and qubit number for all moments
import json
import pandas as pd
# Load JSON data
data = json.loads(json_data)

# Extract gate name, exponent, and qubit number
operations_list = []
for moment in data["moments"]:
    for operation in moment["operations"]:
        gate_name = operation["gate"]["cirq_type"]
        exponent = operation["gate"]["exponent"]
        qubit_numbers = [qubit["x"] for qubit in operation["qubits"]]
        for qubit in qubit_numbers:
            operations_list.append({"Gate": gate_name, "Exponent": exponent, "Qubit": qubit})

# Create DataFrame
df = pd.DataFrame(operations_list)

# Display DataFrame

In [159]:
csv_filename = "gate_operations.csv"
df.to_csv(csv_filename, index=False)

In [167]:
import json

def collect_cz_blocks_with_moments(file_path):
    """
    Same logic as before, but returns blocks with (moment_idx, (q1,q2)) instead of just (q1,q2).
    """
    with open(file_path, "r") as f:
        circuit_json = json.load(f)

    moments = circuit_json["moments"]
    
    # Gather all CZ gates
    cz_list = []
    for m_idx, moment in enumerate(moments):
        for op in moment["operations"]:
            if op["gate"]["cirq_type"] == "CZPowGate":
                qubits_used = [q["x"] for q in op["qubits"]]
                if len(qubits_used) == 2:
                    # store (moment_index, frozenset_of_qubits)
                    cz_list.append((m_idx, frozenset(qubits_used)))
    
    if not cz_list:
        return []

    cz_list.sort(key=lambda x: x[0])

    def has_single_qubit_on(qubits_of_interest, start_m, end_m):
        if start_m > end_m:
            return False
        for m_idx_check in range(start_m, end_m + 1):
            for op in moments[m_idx_check]["operations"]:
                qbs = [q["x"] for q in op["qubits"]]
                if len(qbs) == 1 and qbs[0] in qubits_of_interest:
                    return True
        return False

    blocks = []
    current_block = []  # will hold items of form (moment_idx, (q1, q2))

    # Start with the first
    first_m_idx, first_qubits = cz_list[0]
    current_block.append( (first_m_idx, tuple(first_qubits)) )

    # We'll keep a simple list (for parallel indexing) of moment indices in the block
    for i in range(1, len(cz_list)):
        new_m_idx, new_qubits = cz_list[i]
        
        conflict_found = False
        for (old_m_idx, old_tuple) in current_block:
            old_qubits = frozenset(old_tuple)
            shared = old_qubits.intersection(new_qubits)
            if not shared:
                continue
            if has_single_qubit_on(shared, old_m_idx+1, new_m_idx-1):
                conflict_found = True
                break
        
        if conflict_found:
            blocks.append(current_block)
            current_block = [(new_m_idx, tuple(new_qubits))]
        else:
            current_block.append((new_m_idx, tuple(new_qubits)))

    # finalize
    if current_block:
        blocks.append(current_block)

    return blocks


def extract_single_qubit_ops(moments, start_m, end_m):
    """
    Return a list of single-qubit operations in the inclusive range [start_m, end_m].
    Each element is a dict with info: {"moment": m_idx, "qubit": name, "gate_type": "...", ...}
    """
    sq_ops = []
    if start_m > end_m:
        return sq_ops
    for m_idx in range(start_m, end_m + 1):
        if m_idx < 0 or m_idx >= len(moments):
            continue
        for op in moments[m_idx]["operations"]:
            qbs = [q["x"] for q in op["qubits"]]
            if len(qbs) == 1:
                # single-qubit op
                gate_info = op["gate"]
                sq_ops.append({
                    "moment": m_idx,
                    "qubit": qbs[0],
                    "gate_type": gate_info["cirq_type"],
                    # you could add more details from gate_info if desired
                })
    return sq_ops


def make_command_list(file_path):
    """
    1) Collect blocks (with moment indices).
    2) Interleave each block with the single-qubit ops that occur strictly after the
       block's last moment and before the next block's first moment.
    3) Return a list of commands:
       [
         ("BLOCK", [ (q1,q2), (q3,q4), ... ]),
         ("SINGLE_QUBITS", [ {...}, {...}, ... ]),
         ("BLOCK", ...),
         ...
       ]
    """
    # First parse the JSON
    with open(file_path, "r") as f:
        circuit_json = json.load(f)
    moments = circuit_json["moments"]

    # Get the blocks (with moment indices)
    blocks_with_moments = collect_cz_blocks_with_moments(file_path)
    
    commands = []
    for i, block in enumerate(blocks_with_moments):
        # block is a list of (m_idx, (q1,q2)) 
        # sort the gates in this block by moment index just for clarity
        block_sorted = sorted(block, key=lambda x: x[0])
        # Build a simpler "CZ list" that doesn't store the moment index
        cz_pairs = [x[1] for x in block_sorted]

        # Append the block as a command
        commands.append(("BLOCK", cz_pairs))
        
        # If there's a next block, gather the single-qubit ops in-between
        if i < len(blocks_with_moments) - 1:
            next_block = blocks_with_moments[i+1]
            # find the largest moment in the current block
            max_m_in_block = max(m_idx for (m_idx, _) in block)
            # find the smallest moment in the next block
            min_m_in_next = min(m_idx for (m_idx, _) in next_block)

            # single-qubit gates in (max_m_in_block+1) ... (min_m_in_next - 1)
            sq_ops = extract_single_qubit_ops(moments, max_m_in_block+1, min_m_in_next-1)
            if sq_ops:
                commands.append(("SINGLE_QUBITS", sq_ops))
    
    return commands


if __name__ == "__main__":
    path = "marcelo_3.json"  # replace with your JSON file path
    command_list = make_command_list(path)

    # Print them
    for cmd_type, data in command_list:
        if cmd_type == "BLOCK":
            print(f"BLOCK of CZs: {data}")  # data is a list of (q1,q2) pairs
        elif cmd_type == "SINGLE_QUBITS":
            print("SINGLE_QUBITS between blocks:")
            for sq in data:
                print("  ", sq)

BLOCK of CZs: [(0, 1)]
SINGLE_QUBITS between blocks:
   {'moment': 3, 'qubit': 0, 'gate_type': 'YPowGate'}
   {'moment': 3, 'qubit': 1, 'gate_type': 'ZPowGate'}
   {'moment': 4, 'qubit': 1, 'gate_type': 'YPowGate'}
BLOCK of CZs: [(0, 1)]
SINGLE_QUBITS between blocks:
   {'moment': 6, 'qubit': 0, 'gate_type': 'ZPowGate'}
   {'moment': 6, 'qubit': 1, 'gate_type': 'XPowGate'}
   {'moment': 7, 'qubit': 0, 'gate_type': 'YPowGate'}
   {'moment': 7, 'qubit': 1, 'gate_type': 'ZPowGate'}
BLOCK of CZs: [(0, 3)]
SINGLE_QUBITS between blocks:
   {'moment': 9, 'qubit': 0, 'gate_type': 'ZPowGate'}
   {'moment': 9, 'qubit': 3, 'gate_type': 'ZPowGate'}
   {'moment': 10, 'qubit': 0, 'gate_type': 'YPowGate'}
   {'moment': 10, 'qubit': 3, 'gate_type': 'YPowGate'}
BLOCK of CZs: [(0, 3)]
SINGLE_QUBITS between blocks:
   {'moment': 12, 'qubit': 0, 'gate_type': 'ZPowGate'}
   {'moment': 12, 'qubit': 3, 'gate_type': 'YPowGate'}
   {'moment': 13, 'qubit': 0, 'gate_type': 'YPowGate'}
BLOCK of CZs: [(0, 2)]
SING

In [175]:
import json

def collect_cz_blocks_with_moments(file_path):
    """
    Returns a list of blocks, where each block is a list of (moment_index, (q1,q2)).
    Implements your 'multi-comparison' logic to decide when to start a new block.
    """
    with open(file_path, "r") as f:
        circuit_json = json.load(f)

    moments = circuit_json["moments"]
    
    # Gather all 2-qubit CZs
    cz_list = []
    for m_idx, moment in enumerate(moments):
        for op in moment["operations"]:
            if op["gate"]["cirq_type"] == "CZPowGate":
                qbs = [q["x"] for q in op["qubits"]]
                if len(qbs) == 2:
                    cz_list.append((m_idx, frozenset(qbs)))

    if not cz_list:
        return []

    # Sort by moment index
    cz_list.sort(key=lambda x: x[0])

    def single_qubit_on(qubits_of_interest, start_m, end_m):
        """Return True if in [start_m, end_m] there is a single-qubit gate on a qubit in qubits_of_interest."""
        if start_m > end_m:
            return False
        for check_m in range(start_m, end_m+1):
            if check_m < 0 or check_m >= len(moments):
                continue
            for op in moments[check_m]["operations"]:
                sub_qbs = [qq["x"] for qq in op["qubits"]]
                if len(sub_qbs) == 1 and sub_qbs[0] in qubits_of_interest:
                    return True
        return False

    blocks = []
    current_block = []

    # Start with the first
    first_m_idx, first_set = cz_list[0]
    current_block.append((first_m_idx, tuple(first_set)))  # store tuple for (q1,q2)

    for i in range(1, len(cz_list)):
        new_m_idx, new_set = cz_list[i]
        conflict_found = False
        # Compare new_set with *all* gates in current_block
        for (old_m_idx, old_tuple) in current_block:
            old_set = frozenset(old_tuple)
            shared = old_set.intersection(new_set)
            if shared:
                # check single-qubit gates on 'shared' in the strictly-between region
                if single_qubit_on(shared, old_m_idx+1, new_m_idx-1):
                    conflict_found = True
                    break
        if conflict_found:
            blocks.append(current_block)
            current_block = [(new_m_idx, tuple(new_set))]
        else:
            current_block.append((new_m_idx, tuple(new_set)))

    # finalize
    if current_block:
        blocks.append(current_block)

    return blocks


def extract_single_qubit_ops(moments, start_m, end_m):
    """
    Returns a list of single-qubit gates in inclusive range [start_m, end_m].
    Each entry is a dict with keys: moment, qubit, gate_type, etc.
    """
    if start_m > end_m:
        return []
    results = []
    for m_idx in range(start_m, end_m+1):
        if m_idx < 0 or m_idx >= len(moments):
            continue
        for op in moments[m_idx]["operations"]:
            qbs = [q["x"] for q in op["qubits"]]
            if len(qbs) == 1:
                gate_type = op["gate"]["cirq_type"]
                results.append({
                    "moment": m_idx,
                    "qubit": qbs[0],
                    "gate_type": gate_type,
                    # You can include more fields if you like
                })
    return results


def make_command_list(file_path):
    """
    Interleaves ("BLOCK", [...]) with ("SINGLE_QUBITS", [...]) for the entire circuit.
    Each "BLOCK" item is a list of (moment_index, (q1,q2)).
    Each "SINGLE_QUBITS" item is a list of single-qubit ops (dicts) between blocks.
    """
    with open(file_path, "r") as f:
        circuit_json = json.load(f)
    moments = circuit_json["moments"]

    blocks = collect_cz_blocks_with_moments(file_path)
    commands = []
    for i, block in enumerate(blocks):
        # Sort the gates in the block by moment
        block_sorted = sorted(block, key=lambda x: x[0])
        commands.append(("BLOCK", block_sorted))
        # If there's a next block, gather single-qubit ops in between
        if i < len(blocks) - 1:
            max_m_in_block = max(m_idx for (m_idx, _) in block_sorted)
            next_block = blocks[i+1]
            min_m_in_next = min(m_idx for (m_idx, _) in next_block)
            sq_ops = extract_single_qubit_ops(moments, max_m_in_block+1, min_m_in_next-1)
            if sq_ops:
                commands.append(("SINGLE_QUBITS", sq_ops))
    return commands


def emit_full_program(file_path):
    """
    Produces a *final* list of textual commands that:
      1) Initializes each qubit in a separate gate pair (so no neighbors).
      2) For each 'BLOCK' of CZs, we sequentially move the needed qubits into slots (0,1),
         apply CZ, and move them back.
      3) For each 'SINGLE_QUBITS' list, we just print them out as 'SINGLE qX gate=...'
    """
    # 1) Parse the circuit to get a full list of qubits
    with open(file_path, "r") as f:
        circuit_json = json.load(f)
    moments = circuit_json["moments"]

    # Gather every qubit name mentioned in the circuit
    all_qubits = set()
    for m_idx, moment in enumerate(moments):
        for op in moment["operations"]:
            for qq in op["qubits"]:
                all_qubits.add(qq["x"])
    all_qubits = sorted(all_qubits)  # e.g. ['q_0','q_1','q_2',...]

    # We will place qubit i in "slot = 2*i" so that the neighbor slot = 2*i+1 is empty.
    # Store a dictionary for their "home" slot
    home_slot = {}
    for i, qb in enumerate(all_qubits):
        home_slot[qb] = 2*i

    # 2) Build the command list (blocks + single-qubit sections)
    cmd_list = make_command_list(file_path)

    # We'll accumulate text lines in 'program_lines'
    program_lines = []

    # 3) "INIT" lines: place each qubit in its home slot
    for qb in all_qubits:
        slot = home_slot[qb]
        program_lines.append(f"INIT {qb} -> slot{slot}")

    # 4) Then walk through cmd_list in order
    for (cmd_type, data) in cmd_list:
        if cmd_type == "BLOCK":
            # data is a list of (m_idx, (q1,q2)) gates
            for (m_idx, (q1, q2)) in data:
                # We'll do the "MOVE q1 -> slot0" / "MOVE q2 -> slot1" etc.
                original_slot_q1 = home_slot[q1]
                original_slot_q2 = home_slot[q2]

                program_lines.append(f"MOVE {q1} -> slot0")
                program_lines.append(f"MOVE {q2} -> slot1")

                program_lines.append(f"CZ({q1},{q2})")

                # Move them back
                program_lines.append(f"MOVE {q1} -> slot{original_slot_q1}")
                program_lines.append(f"MOVE {q2} -> slot{original_slot_q2}")

        elif cmd_type == "SINGLE_QUBITS":
            # data is a list of single-qubit op dicts
            for op_info in data:
                qb = op_info["qubit"]
                gtype = op_info["gate_type"]
                moment = op_info["moment"]
                # Just print something like:
                program_lines.append(f"SINGLE {qb}, gate={gtype}, moment={moment}")

    return program_lines


# ------------------ Example usage ------------------
if __name__ == "__main__":
    path = "marcelo_3.json"  # Replace with your actual JSON file
    final_lines = emit_full_program(path)

    # Print them out
    print("===== FINAL COMMAND SEQUENCE =====")
    for line in final_lines:
        print(line)

===== FINAL COMMAND SEQUENCE =====
INIT 0 -> slot0
INIT 1 -> slot2
INIT 2 -> slot4
INIT 3 -> slot6
MOVE 0 -> slot0
MOVE 1 -> slot1
CZ(0,1)
MOVE 0 -> slot0
MOVE 1 -> slot2
SINGLE 0, gate=YPowGate, moment=3
SINGLE 1, gate=ZPowGate, moment=3
SINGLE 1, gate=YPowGate, moment=4
MOVE 0 -> slot0
MOVE 1 -> slot1
CZ(0,1)
MOVE 0 -> slot0
MOVE 1 -> slot2
SINGLE 0, gate=ZPowGate, moment=6
SINGLE 1, gate=XPowGate, moment=6
SINGLE 0, gate=YPowGate, moment=7
SINGLE 1, gate=ZPowGate, moment=7
MOVE 0 -> slot0
MOVE 3 -> slot1
CZ(0,3)
MOVE 0 -> slot0
MOVE 3 -> slot6
SINGLE 0, gate=ZPowGate, moment=9
SINGLE 3, gate=ZPowGate, moment=9
SINGLE 0, gate=YPowGate, moment=10
SINGLE 3, gate=YPowGate, moment=10
MOVE 0 -> slot0
MOVE 3 -> slot1
CZ(0,3)
MOVE 0 -> slot0
MOVE 3 -> slot6
SINGLE 0, gate=ZPowGate, moment=12
SINGLE 3, gate=YPowGate, moment=12
SINGLE 0, gate=YPowGate, moment=13
MOVE 0 -> slot0
MOVE 2 -> slot1
CZ(0,2)
MOVE 0 -> slot0
MOVE 2 -> slot4
SINGLE 0, gate=ZPowGate, moment=15
SINGLE 2, gate=ZPowGate, 

In [177]:
import json

def collect_cz_blocks_with_moments(file_path):
    """
    Returns a list of blocks, where each block is a list of (moment_index, (q1,q2)).
    Implements your 'multi-comparison' logic to decide when to start a new block.
    """
    with open(file_path, "r") as f:
        circuit_json = json.load(f)

    moments = circuit_json["moments"]
    
    # Gather all 2-qubit CZs
    cz_list = []
    for m_idx, moment in enumerate(moments):
        for op in moment["operations"]:
            if op["gate"]["cirq_type"] == "CZPowGate":
                qbs = [q["x"] for q in op["qubits"]]
                if len(qbs) == 2:
                    cz_list.append((m_idx, frozenset(qbs)))

    if not cz_list:
        return []

    # Sort by moment index
    cz_list.sort(key=lambda x: x[0])

    def single_qubit_on(qubits_of_interest, start_m, end_m):
        """Return True if in [start_m, end_m] there is a single-qubit gate on a qubit in qubits_of_interest."""
        if start_m > end_m:
            return False
        for check_m in range(start_m, end_m+1):
            if check_m < 0 or check_m >= len(moments):
                continue
            for op in moments[check_m]["operations"]:
                sub_qbs = [qq["x"] for qq in op["qubits"]]
                if len(sub_qbs) == 1 and sub_qbs[0] in qubits_of_interest:
                    return True
        return False

    blocks = []
    current_block = []

    # Start with the first
    first_m_idx, first_set = cz_list[0]
    current_block.append((first_m_idx, tuple(first_set)))  # store tuple for (q1,q2)

    for i in range(1, len(cz_list)):
        new_m_idx, new_set = cz_list[i]
        conflict_found = False
        # Compare new_set with *all* gates in current_block
        for (old_m_idx, old_tuple) in current_block:
            old_set = frozenset(old_tuple)
            shared = old_set.intersection(new_set)
            if shared:
                # check single-qubit gates on 'shared' in the strictly-between region
                if single_qubit_on(shared, old_m_idx+1, new_m_idx-1):
                    conflict_found = True
                    break
        if conflict_found:
            blocks.append(current_block)
            current_block = [(new_m_idx, tuple(new_set))]
        else:
            current_block.append((new_m_idx, tuple(new_set)))

    # finalize
    if current_block:
        blocks.append(current_block)

    return blocks


def extract_single_qubit_ops(moments, start_m, end_m):
    """
    Returns a list of single-qubit gates in inclusive range [start_m, end_m].
    Each entry is a dict with keys: moment, qubit, gate_type, etc.
    """
    if start_m > end_m:
        return []
    results = []
    for m_idx in range(start_m, end_m+1):
        if m_idx < 0 or m_idx >= len(moments):
            continue
        for op in moments[m_idx]["operations"]:
            qbs = [q["x"] for q in op["qubits"]]
            if len(qbs) == 1:
                gate_type = op["gate"]["cirq_type"]
                results.append({
                    "moment": m_idx,
                    "qubit": qbs[0],
                    "gate_type": gate_type,
                })
    return results


def make_command_list(file_path):
    """
    Interleaves ("BLOCK", [...]) with ("SINGLE_QUBITS", [...]) for the entire circuit.
    Each "BLOCK" item is a list of (moment_index, (q1,q2)).
    Each "SINGLE_QUBITS" item is a list of single-qubit ops (dicts) between blocks.
    """
    with open(file_path, "r") as f:
        circuit_json = json.load(f)
    moments = circuit_json["moments"]

    blocks = collect_cz_blocks_with_moments(file_path)
    commands = []
    for i, block in enumerate(blocks):
        block_sorted = sorted(block, key=lambda x: x[0])
        commands.append(("BLOCK", block_sorted))
        if i < len(blocks) - 1:
            max_m_in_block = max(m_idx for (m_idx, _) in block_sorted)
            next_block = blocks[i+1]
            min_m_in_next = min(m_idx for (m_idx, _) in next_block)
            sq_ops = extract_single_qubit_ops(moments, max_m_in_block+1, min_m_in_next-1)
            if sq_ops:
                commands.append(("SINGLE_QUBITS", sq_ops))
    return commands


def emit_full_program(file_path):
    """
    Produces a *final* list of textual commands that:
      1) Initializes each qubit in a separate gate pair (so no neighbors).
      2) For each 'BLOCK' of CZs, we sequentially move the needed qubits into slots (0,1),
         apply CZ, and move them back -- but skip any MOVE if the qubit is already in that slot.
      3) For each 'SINGLE_QUBITS' list, we just print them out as 'SINGLE qX gate=...'
    """
    # 1) Parse the circuit to get a full list of qubits
    with open(file_path, "r") as f:
        circuit_json = json.load(f)
    moments = circuit_json["moments"]

    # Gather every qubit name
    all_qubits = set()
    for m_idx, moment in enumerate(moments):
        for op in moment["operations"]:
            for qq in op["qubits"]:
                all_qubits.add(qq["x"])
    all_qubits = sorted(all_qubits)  # e.g. ['q_0','q_1','q_2',...]

    # Assign each qubit a "home" slot = 2*i
    home_slot = {}
    for i, qb in enumerate(all_qubits):
        home_slot[qb] = 2*i

    # We'll track each qubit's *current* slot in a dict
    current_slot = dict(home_slot)  # initially at home slot

    # 2) Build the command list (blocks + single-qubit sections)
    cmd_list = make_command_list(file_path)

    program_lines = []

    # 3) "INIT" lines
    for qb in all_qubits:
        slot = home_slot[qb]
        program_lines.append(f"INIT {qb} -> slot{slot}")

    # Helper function to do a conditional move
    def move_qubit_if_needed(qb, target_slot):
        """Emit a MOVE command only if qb is not already in target_slot."""
        if current_slot[qb] != target_slot:
            program_lines.append(f"MOVE {qb} -> slot{target_slot}")
            current_slot[qb] = target_slot

    # 4) Walk through cmd_list
    for (cmd_type, data) in cmd_list:
        if cmd_type == "BLOCK":
            # data is a list of (m_idx, (q1,q2)) gates
            for (m_idx, (q1, q2)) in data:
                # Original slots
                orig_slot_q1 = home_slot[q1]
                orig_slot_q2 = home_slot[q2]

                # Move to (0,1) if needed
                move_qubit_if_needed(q1, 0)
                move_qubit_if_needed(q2, 1)

                # Emit CZ
                program_lines.append(f"CZ({q1},{q2})")

                # Move them back if needed
                move_qubit_if_needed(q1, orig_slot_q1)
                move_qubit_if_needed(q2, orig_slot_q2)

        elif cmd_type == "SINGLE_QUBITS":
            for op_info in data:
                qb = op_info["qubit"]
                gtype = op_info["gate_type"]
                moment = op_info["moment"]
                program_lines.append(f"SINGLE {qb}, gate={gtype}, moment={moment}")

    return program_lines


# ------------------ Example usage ------------------
if __name__ == "__main__":
    path = "marcelo_3.json"  # Replace with your actual JSON file
    final_lines = emit_full_program(path)

    print("===== FINAL COMMAND SEQUENCE =====")
    for line in final_lines:
        print(line)

===== FINAL COMMAND SEQUENCE =====
INIT 0 -> slot0
INIT 1 -> slot2
INIT 2 -> slot4
INIT 3 -> slot6
MOVE 1 -> slot1
CZ(0,1)
MOVE 1 -> slot2
SINGLE 0, gate=YPowGate, moment=3
SINGLE 1, gate=ZPowGate, moment=3
SINGLE 1, gate=YPowGate, moment=4
MOVE 1 -> slot1
CZ(0,1)
MOVE 1 -> slot2
SINGLE 0, gate=ZPowGate, moment=6
SINGLE 1, gate=XPowGate, moment=6
SINGLE 0, gate=YPowGate, moment=7
SINGLE 1, gate=ZPowGate, moment=7
MOVE 3 -> slot1
CZ(0,3)
MOVE 3 -> slot6
SINGLE 0, gate=ZPowGate, moment=9
SINGLE 3, gate=ZPowGate, moment=9
SINGLE 0, gate=YPowGate, moment=10
SINGLE 3, gate=YPowGate, moment=10
MOVE 3 -> slot1
CZ(0,3)
MOVE 3 -> slot6
SINGLE 0, gate=ZPowGate, moment=12
SINGLE 3, gate=YPowGate, moment=12
SINGLE 0, gate=YPowGate, moment=13
MOVE 2 -> slot1
CZ(0,2)
MOVE 2 -> slot4
SINGLE 0, gate=ZPowGate, moment=15
SINGLE 2, gate=ZPowGate, moment=15
SINGLE 0, gate=YPowGate, moment=16
SINGLE 2, gate=YPowGate, moment=16
MOVE 2 -> slot1
CZ(0,2)
MOVE 2 -> slot4
SINGLE 0, gate=ZPowGate, moment=18
SINGL

In [183]:
import json

def collect_cz_blocks_with_moments(file_path):
    """
    Returns a list of blocks, where each block is a list of (moment_index, (q1,q2)).
    Implements your 'multi-comparison' logic to decide when to start a new block.
    """
    with open(file_path, "r") as f:
        circuit_json = json.load(f)

    moments = circuit_json["moments"]
    
    # Gather all 2-qubit CZs
    cz_list = []
    for m_idx, moment in enumerate(moments):
        for op in moment["operations"]:
            if op["gate"]["cirq_type"] == "CZPowGate":
                qbs = [q["x"] for q in op["qubits"]]
                if len(qbs) == 2:
                    cz_list.append((m_idx, frozenset(qbs)))

    if not cz_list:
        return []

    # Sort by moment index
    cz_list.sort(key=lambda x: x[0])

    def single_qubit_on(qubits_of_interest, start_m, end_m):
        """Return True if in [start_m, end_m] there's a single-qubit gate on any of qubits_of_interest."""
        if start_m > end_m:
            return False
        for check_m in range(start_m, end_m+1):
            if check_m < 0 or check_m >= len(moments):
                continue
            for op in moments[check_m]["operations"]:
                sub_qbs = [qq["x"] for qq in op["qubits"]]
                if len(sub_qbs) == 1 and sub_qbs[0] in qubits_of_interest:
                    return True
        return False

    blocks = []
    current_block = []

    # Start with the first
    first_m_idx, first_set = cz_list[0]
    current_block.append((first_m_idx, tuple(first_set)))  # store tuple for (q1,q2)

    for i in range(1, len(cz_list)):
        new_m_idx, new_set = cz_list[i]
        conflict_found = False
        # Compare new_set with *all* gates in current_block
        for (old_m_idx, old_tuple) in current_block:
            old_set = frozenset(old_tuple)
            shared = old_set.intersection(new_set)
            if shared:
                # check single-qubit gates on 'shared' in strictly-between region
                if single_qubit_on(shared, old_m_idx+1, new_m_idx-1):
                    conflict_found = True
                    break
        if conflict_found:
            blocks.append(current_block)
            current_block = [(new_m_idx, tuple(new_set))]
        else:
            current_block.append((new_m_idx, tuple(new_set)))

    # finalize
    if current_block:
        blocks.append(current_block)

    return blocks


def extract_single_qubit_ops(moments, start_m, end_m):
    """
    Returns a list of single-qubit gates in inclusive range [start_m, end_m].
    Each entry is a dict with keys: moment, qubit, gate_type, etc.
    """
    if start_m > end_m:
        return []
    results = []
    for m_idx in range(start_m, end_m+1):
        if m_idx < 0 or m_idx >= len(moments):
            continue
        for op in moments[m_idx]["operations"]:
            qbs = [q["x"] for q in op["qubits"]]
            if len(qbs) == 1:
                gate_type = op["gate"]["cirq_type"]
                results.append({
                    "moment": m_idx,
                    "qubit": qbs[0],
                    "gate_type": gate_type,
                })
    return results


def make_command_list(file_path):
    """
    Interleaves ("BLOCK", [...]) with ("SINGLE_QUBITS", [...]) for the entire circuit.
    Each "BLOCK" item is a list of (moment_index, (q1,q2)).
    Each "SINGLE_QUBITS" item is a list of single-qubit ops (dicts) between blocks.
    """
    with open(file_path, "r") as f:
        circuit_json = json.load(f)
    moments = circuit_json["moments"]

    blocks = collect_cz_blocks_with_moments(file_path)
    commands = []
    for i, block in enumerate(blocks):
        block_sorted = sorted(block, key=lambda x: x[0])
        commands.append(("BLOCK", block_sorted))
        if i < len(blocks) - 1:
            max_m_in_block = max(m_idx for (m_idx, _) in block_sorted)
            next_block = blocks[i+1]
            min_m_in_next = min(m_idx for (m_idx, _) in next_block)
            sq_ops = extract_single_qubit_ops(moments, max_m_in_block+1, min_m_in_next-1)
            if sq_ops:
                commands.append(("SINGLE_QUBITS", sq_ops))
    return commands


def emit_full_program(file_path):
    """
    Produces a *final* list of textual commands that:
      1) Initializes each qubit in a separate gate pair (so no neighbors).
      2) For each 'BLOCK' of CZs, we sequentially move the needed qubits into slots (0,1),
         apply CZ, and move them back -- skipping any MOVE if the qubit is already in that slot.
         The lines are of the form:
           MOVE q3 from slot6 to slot1 -> q3 q0
      3) For each 'SINGLE_QUBITS' list, we just print them out as 'SINGLE qX gate=...'
    """
    import json

    with open(file_path, "r") as f:
        circuit_json = json.load(f)
    moments = circuit_json["moments"]

    # Gather every qubit name (which might be strings or ints in your JSON)
    all_qubits = set()
    for m_idx, moment in enumerate(moments):
        for op in moment["operations"]:
            for qq in op["qubits"]:
                all_qubits.add(qq["x"])
    # Convert to a sorted list for consistency
    all_qubits = sorted(all_qubits, key=lambda x: str(x))  # sort by string form

    def format_qubit_name(qb_name):
        qb_name_str = str(qb_name)  # Ensure it's a string
        if qb_name_str.startswith("q_"):
            # e.g. 'q_3' => 'q3'
            return "q" + qb_name_str[2:]
        else:
            # e.g. '3' => 'q3', or 'abc' => 'qabc'
            return "q" + qb_name_str

    # Home slot is 2*i for each qubit
    home_slot = {}
    for i, qb in enumerate(all_qubits):
        home_slot[qb] = 2*i

    current_slot = dict(home_slot)

    cmd_list = make_command_list(file_path)

    program_lines = []

    # Initialize lines
    for qb in all_qubits:
        slot = home_slot[qb]
        qb_formatted = format_qubit_name(qb)
        program_lines.append(f"INIT {qb_formatted} -> slot{slot}")

    def move_qubit_if_needed(qb, target_slot, partner_qb=None):
        current = current_slot[qb]
        if current != target_slot:
            qb_label = format_qubit_name(qb)
            from_slot = current
            to_slot = target_slot
            if partner_qb is None:
                partner_label = "0"
            else:
                partner_label = format_qubit_name(partner_qb)

            line = f"MOVE {qb_label} from slot{from_slot} to slot{to_slot} -> {qb_label} {partner_label}"
            program_lines.append(line)
            current_slot[qb] = target_slot

    # Emit the BLOCK or SINGLE_QUBITS commands
    for (cmd_type, data) in cmd_list:
        if cmd_type == "BLOCK":
            for (m_idx, (q1, q2)) in data:
                orig_q1_slot = home_slot[q1]
                orig_q2_slot = home_slot[q2]

                move_qubit_if_needed(q1, 0, partner_qb=q2)
                move_qubit_if_needed(q2, 1, partner_qb=q1)

                q1f = format_qubit_name(q1)
                q2f = format_qubit_name(q2)
                program_lines.append(f"CZ({q1f},{q2f})")

                move_qubit_if_needed(q1, orig_q1_slot, partner_qb=q2)
                move_qubit_if_needed(q2, orig_q2_slot, partner_qb=q1)

        elif cmd_type == "SINGLE_QUBITS":
            for op_info in data:
                qb = op_info["qubit"]
                gtype = op_info["gate_type"]
                moment = op_info["moment"]
                qb_label = format_qubit_name(qb)
                program_lines.append(f"SINGLE {qb_label}, gate={gtype}, moment={moment}")

    return program_lines


# ------------------ Example usage ------------------
if __name__ == "__main__":
    path = "marcelo_3.json"  # Replace with your actual JSON file
    final_lines = emit_full_program(path)

    print("===== FINAL COMMAND SEQUENCE =====")
    for line in final_lines:
        print(line)

===== FINAL COMMAND SEQUENCE =====
INIT q0 -> slot0
INIT q1 -> slot2
INIT q2 -> slot4
INIT q3 -> slot6
MOVE q1 from slot2 to slot1 -> q1 q0
CZ(q0,q1)
MOVE q1 from slot1 to slot2 -> q1 q0
SINGLE q0, gate=YPowGate, moment=3
SINGLE q1, gate=ZPowGate, moment=3
SINGLE q1, gate=YPowGate, moment=4
MOVE q1 from slot2 to slot1 -> q1 q0
CZ(q0,q1)
MOVE q1 from slot1 to slot2 -> q1 q0
SINGLE q0, gate=ZPowGate, moment=6
SINGLE q1, gate=XPowGate, moment=6
SINGLE q0, gate=YPowGate, moment=7
SINGLE q1, gate=ZPowGate, moment=7
MOVE q3 from slot6 to slot1 -> q3 q0
CZ(q0,q3)
MOVE q3 from slot1 to slot6 -> q3 q0
SINGLE q0, gate=ZPowGate, moment=9
SINGLE q3, gate=ZPowGate, moment=9
SINGLE q0, gate=YPowGate, moment=10
SINGLE q3, gate=YPowGate, moment=10
MOVE q3 from slot6 to slot1 -> q3 q0
CZ(q0,q3)
MOVE q3 from slot1 to slot6 -> q3 q0
SINGLE q0, gate=ZPowGate, moment=12
SINGLE q3, gate=YPowGate, moment=12
SINGLE q0, gate=YPowGate, moment=13
MOVE q2 from slot4 to slot1 -> q2 q0
CZ(q0,q2)
MOVE q2 from slot1 